1. In class we wrote code to get all of the links from the G77 statements website: https://www.g77.org/statement/index.php
Please write code that iterates through each of the links and formats them properly so that they are a readable url. Print them to the console.



In [11]:
import requests
from bs4 import BeautifulSoup

url = "https://www.g77.org/statement"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a")
    # Create an empty list for all the URLs
    url_list = []
    # Extract and print the href attributes from the links
    for link in links:
        href = link.get("href")
        if href and href != "javascript:;": # Ignore JavaScript links
            if href[0] == "#":
                url_str = "https://www.g77.org/statement/" + href
            else:
                url_str = "https://www.g77.org" + href
            
            print(url_str)
            url_list.append(url_str)
else:
    print(f"Failed to retrieve content: HTTP {response.status_code}")

https://www.g77.org/statement/#jan
https://www.g77.org/statement/#jul
https://www.g77.org/statement/#feb
https://www.g77.org/statement/#aug
https://www.g77.org/statement/#mar
https://www.g77.org/statement/#sep
https://www.g77.org/statement/#apr
https://www.g77.org/statement/#oct
https://www.g77.org/statement/#may
https://www.g77.org/statement/#nov
https://www.g77.org/statement/#jun
https://www.g77.org/statement/#dec
https://www.g77.org/statement/getstatement.php?id=240215
https://www.g77.org/statement/getstatement.php?id=240208
https://www.g77.org/statement/getstatement.php?id=240207b
https://www.g77.org/statement/getstatement.php?id=240207
https://www.g77.org/statement/getstatement.php?id=240206b
https://www.g77.org/statement/getstatement.php?id=240206
https://www.g77.org/statement/getstatement.php?id=240205c
https://www.g77.org/statement/getstatement.php?id=240205b
https://www.g77.org/statement/getstatement.php?id=240205
https://www.g77.org/statement/getstatement.php?id=240202
https:

2. Now modify your code so that the urls are saved as a list. Iterate through the list, open each url, and extract the text from each, and save the text to a file called G77_2024.

In [23]:
with open("G77_2024.txt", "w") as file:
    for url in url_list:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")
            statement_text = "\n".join([paragraph.text.strip() for paragraph in paragraphs])
            # Write to file only if there's meaningful content
            if statement_text:
                file.write(statement_text + "\n\n---\n\n")
            else:
                print(f"No meaningful content found in {url}")
        else:
            print(f"Failed to retrieve content from {url}: HTTP {resp.status_code}")

No meaningful content found in https://www.g77.org/statement/#jan
No meaningful content found in https://www.g77.org/statement/#jul
No meaningful content found in https://www.g77.org/statement/#feb
No meaningful content found in https://www.g77.org/statement/#aug
No meaningful content found in https://www.g77.org/statement/#mar
No meaningful content found in https://www.g77.org/statement/#sep
No meaningful content found in https://www.g77.org/statement/#apr
No meaningful content found in https://www.g77.org/statement/#oct
No meaningful content found in https://www.g77.org/statement/#may
No meaningful content found in https://www.g77.org/statement/#nov
No meaningful content found in https://www.g77.org/statement/#jun
No meaningful content found in https://www.g77.org/statement/#dec
No meaningful content found in https://www.g77.org/statement/#
No meaningful content found in https://www.g77.org/statement/#


3. Perform a word frequency distribution on the text and visualize the top 20 words used in these statements.( Please remove any stopwords first)

In [ ]:
#Your code here:

4. Here is a link to a speech made by President Trump on January 6: https://www.npr.org/2021/02/10/966396848/read-trumps-jan-6-speech-a-key-part-of-impeachment-trial. Not all of the text is his speech. Some of the text is the analysis. Please extract Donald Trumps speech only, remove stopwords and perform a word frequency distribution and visualize it. 

In [ ]:
#Your code here:


5. Here is a link to a github repo that contains Donald Trump's speeches: https://github.com/ryanmcdermott/trump-speeches/blob/master/speeches.txt

What are the 10 most common things Donald Trump "loves?"



In [ ]:
#Your code here:

In [ ]:
6. Which are the top 5 countries Trump mentions in his speeches, besides America or the United States of America?

In [ ]:
#Your code here:

In class we talked about sentence tokenizers that could be useful in determining when sentences might be repeated. One application we discussed was in political speeches, when certain sentences or phrases are repeated to show a candidates coaching, focus or in marketing terms, "messaging." Here is an analyis from https://www.vox.com/2016/8/18/12423688/donald-trump-speech-style-explained-by-linguists of the "salesman" techniques that Trump uses:
"Trump’s speeches can be appealing because he uses a lot of salesmen’s tricks. Lakoff, for his part, has an explanation for why Trump’s style of speaking is so appealing to many. Many of Trump’s most famous catchphrases are actually versions of time-tested speech mechanisms that salesmen use. They’re powerful because they help shape our unconscious. Take, for example, Trump’s frequent use of "Many people are saying..." or "Believe me" — often right after saying something that is baseless or untrue. This tends to sound more trustworthy to listeners than just outright stating the baseless claim, since Trump implies that he has direct experience with what he’s talking about. At a base level, Lakoff argues, people are more inclined to believe something that seems to have been shared. Or when Trump keeps calling Clinton "crooked," or keeps referring to terrorists as "radical Muslims," he’s strengthening the association through repetition. He also calls his supporters "folks," to show he is one of them (though many politicians employ this trick). Trump doesn’t repeat phrases and adjectives because he is stalling for time, Liberman says; for the most part, he’s providing emphasis and strengthening the association.
These are normal techniques, particularly in conversational speech. "Is he reading cognitive science? No. He has 50 years of experience as a salesman who doesn’t care who he is selling to," Lakoff says. On this account, Trump uses similar methods in his QVC-style pitch of steaks and vodka as when he talks about his plan to stop ISIS."He has been doing this for a very long time as a salesman — that’s what he is best at," Lakoff says."


7. Perform a frequency analysis that provides evidence for the assertion made in the Vox article. 

Consider tokenizing by unigram (one word), bigram(two words), trigram(three words) or more, or whole sentences or multiple approaches that help us understand the most common Trump linguistic characteristcs. Use your evidence and words to describe what you found. This is a fairly open ask. Don't just execute code. Tell me and show me something interesting!


In [ ]:
#Your code here:

8.Sentiment Analysis
Sentiment analysis, also known as opinion mining or emotion AI, is a field of natural language processing (NLP) that focuses on identifying and categorizing opinions or sentiments expressed within text data. The primary goal is to determine the writer's or speaker's attitude towards a particular topic, product, service, or overall context. This attitude can range from positive, negative, to neutral, and may also encompass more nuanced emotions like happiness, anger, sadness, quantitative or emotional.


The most basic form of sentiment analysis assigns values to words based on a dictionary of words, from neutral to slightly positive or negative, moderately positive or negative, and extremely positive or negative. Vader is a popular package that analyzes sentiment.

In [12]:
#run this
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 144.4 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip



This will compute a compound score by summing the valence scores of each word int he lexicon and then bnormalized between -1 (most negative) and +1 most positive. This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a "normalize, weighted composite score" is accurate.

let's test opur first sentiment using VADER. VADER is great on social media data which can be messy and contain emojis.
We will use the polarity_scores( ) method to obtain the polarity indices.

In [20]:
sentence = "I love that movie!"
score = analyzer.polarity_scores(sentence)
print(score)

{'neg': 0.0, 'neu': 0.4, 'pos': 0.6, 'compound': 0.6696}


In [21]:
sentence2 = "OMG, this job totally SUX!!"
score = analyzer.polarity_scores(sentence2)
print(score)

{'neg': 0.507, 'neu': 0.493, 'pos': 0.0, 'compound': -0.6261}


In [23]:
sentence3 = '😀' #command+control+space brings up emoji'
score = analyzer.polarity_scores(sentence3)
print(score)

{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}


In [24]:
sentence4 = '😀 😀 😀 😀 😀' #command+control+space brings up emoji'
score = analyzer.polarity_scores(sentence4)
print(score)

{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.8885}


The global financial crisis (GFC) refers to the period of extreme stress in global financial markets and banking systems between mid 2007 and early 2009. During the GFC, a downturn in the US housing market was a catalyst for a financial crisis that spread from the United States to the rest of the world through linkages in the global financial system. Many banks around the world incurred large losses and relied on government support to avoid bankruptcy. Millions of people lost their jobs as the major advanced economies experienced their deepest recessions since the Great Depression in the 1930s.

The Federal Reserve conference calls are a good way to track the sentiment that the Fed had toward the crisis. Did they know how bad it was? Did they think they could fix it? Starting in January of 2008, ending in October of 2008 map the sentiment of the Fed reserve calls.

Here are your documents:
Jan 2008: https://www.federalreserve.gov/monetarypolicy/files/FOMC20080121confcall.pdf
Mar 2008: https://www.federalreserve.gov/monetarypolicy/files/FOMC20080310confcall.pdf
July 2008: https://www.federalreserve.gov/monetarypolicy/files/FOMC20080724confcall.pdf
Sept 2008: https://www.federalreserve.gov/monetarypolicy/files/FOMC20080929confcall.pdf
Oct 2008: https://www.federalreserve.gov/monetarypolicy/files/FOMC20081007confcall.pdf


You will need to convert these pdf into text to process. 
1. Perform a word frequency analysis of each of the calls. Don't forget to remove stopwords.
2. Find and present any evidence that the Fed understands that what the US and world is going through is unlike anything that has ever before been experienced. What are the key phrases that convey this idea? How do you find them? Can you use a vector based approach or would you use a custom dictionary?
3. Extract any evidence that they thought that the situation could be quickly remedied and would not get as bad as it ended up gettting.
4. Parse the key participants and plot their sentiment over time. Who is the most positive? Who is the most negative? Who changes the most?
5. Add key makers to your visualization that bring in the key events of 2008.


In [ ]:
#Your code here

9. Sentiment analysis is not a perfect science, especially when you are using off-the-shelf packages like VADER. Given what you know about the crisis, do you trust Vader's sentiment analysis? WHy or why not?



#Your answer here:


10. Can you reasonably determine whether a low or negative compound score indicates a negative sentiment from the Fed in 2008? Can you do so with data only from 2008?

Provide a reasonable comparison from 2008 values by comparing it to some other timeframe from the FED confernce call historic database. https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm

In [ ]:
#Your code here: